# Census Data to Indicators



In [1]:
# NEED TO TAKE LAST FOUR DIGITS OF KALCODE TO LINK TO OA

### R Libraries

The relvant R libraries are imported in to the kernal:

In [2]:
# libraries
# Load R libraries
 if(!require("pacman"))
     install.packages("pacman")
     library("pacman")

p_load("dplyr", "sf", "purrr", "tidyverse", "xlsx")

print("Loaded Packages:")
p_loaded()

Loading required package: pacman



[1] "Loaded Packages:"


[1] "xlsx"      "lubridate" "forcats"   "stringr"   "readr"     "tidyr"    
 [7] "tibble"    "ggplot2"   "tidyverse" "purrr"     "sf"        "dplyr"    
[13] "pacman"

In [3]:
# create the pipeline directory if it does not exist
pipelineDir <- file.path("../..","2_pipeline","Athens","1a_CensusData","2011")
if(!dir.exists(pipelineDir)){
    dir.create(pipelineDir, recursive = TRUE)
    print(paste0(pipelineDir, " created"))
}

## create the export directory if it does not exist
#exportDir <- file.path("../..","0_data","Athens","GreekCensus", "2011","english","processed")
#if(!dir.exists(exportDir)){
#    dir.create(exportDir, recursive = TRUE)
#    print(paste0(exportDir, " created"))
#}
##exportDir <- "../0_data/Athens/GreekCensus/2011/english/processed/"

## Census Data

The Greek census data is available from https://www.statistics.gr/el/2011-census-pop-hous. Don't use the English version of the website as not all the data is displayed - just use Google Translate version. The data used is at the 'Municipality' level, this is the smallest spatial resolution available. These are the data that have the 'B' prefix.

The data is available here:

- Demographic characteristics / 2011 https://www.statistics.gr/el/statistics/-/publication/SAM03/2011
- Characteristics of houses - households / 2011 https://www.statistics.gr/el/statistics/-/publication/SAM05/2011
- Economic characteristics / 2011 https://www.statistics.gr/el/statistics/-/publication/SAM04/2011
- The spatial data is available from [http://gisc.gr/en/data/](http://gisc.gr/en/data/)


## Processing of Census Data

The Greek 2011 Census is in a number of csvs, in Greek, with variations in the data structure. The files need to be translated and processed into the same format. that need to be combined into a single dataset

In [4]:
# need to convert to xlsx so they can be translated
files <- list.files("../../0_data/Athens/GreekCensus/2011/english/", pattern = ".xlsx$", full.names = TRUE,)

# files are then translated one by one with google translate

In [5]:
# # get the files in a standard format - only need to run this once
# for(i in files){
    
#     # remove fieldNames
#     if(exists("fieldNames")){
#         remove("fieldNames")
#     }

#     file <- read.xlsx(file = i, 1, check.names = FALSE, endRow = 10)
    
#     #get the table name summary
#     tableName <- names(file)[1]
    
#     print(paste0('Processing: ', tableName))
#     print(basename(i))
    
#     # get the row the data starts on
#     #function
#     matches <- apply(file,1,function(x) sum(grepl("TOTAL COUNTRY",x)))>0
    
#     dataStartRow <- as.numeric(row.names(file[matches,]))
#     print(paste0('Data starts on row: ', dataStartRow))
#     flush.console()
    
#     #get the number of columns that are not all na
#     dataEndCol <- ncol(file[, colSums(is.na(file)) != nrow(file)])
    
#     #check all cols are not NA
#     noNAColCheck <- FALSE
    
#     while(noNAColCheck == FALSE){
#         # read the data
#         file <- read.xlsx(file = i, 1, check.names = FALSE, endRow = dataStartRow-1, colIndex = c(1:dataEndCol))
        
#         # a row to the start row
#         dataStartRow <- dataStartRow +1
#         print(dataStartRow)
        
#         # check to see if any columns are NA
#         noNAColCheck <- length(as.vector(which(colSums(is.na(file)) == nrow(file)))) == 0
#     }
        
#     # check to see if all columns have data
#     if(length(as.vector(which(colSums(is.na(file)) == nrow(file)))) > 0){
        
#     }
                     
#     for(col in 1:ncol(file)){
#         row <- dataStartRow-1
#         cell <- NA
#         while(is.na(cell)){
#             cell <- as.character(file[row,col])
#             row <- row-1
#         }      

# #         print(paste0("Field name: ", cell))

#         # make a vector of fieldnames
#         if(!exists("fieldNames")){
#             fieldNames <- c(as.character(cell))
#         } else {
#             fieldNames <- append(fieldNames, cell )
#         }


#     }
    
#     # ingest data and give the correct field names
#     file <- read.xlsx(file = i, 1, check.names = FALSE, startRow = dataStartRow-1, colIndex = c(1:dataEndCol))
#     names(file) <- fieldNames 
    
#     # shorten the name of the file 
#     splitName <- str_trim(str_split(tableName, patter="\\.", simplify = TRUE))
    
#     # create the export file name
#     fileNameExport <- paste0(sub(pattern = "(.*)\\..*$", replacement = "\\1", basename(i)), "_", paste(splitName[2:length(splitName)], collapse="_"))
#     print(fileNameExport)
#     flush.console()
                     
#     #export
#     write.csv(file, paste0(exportDir,  str_trunc(fileNameExport, 175, "right"), ".csv"), row.names=FALSE)
#     print(paste0(exportDir, fileNameExport))
# }

### Import Census Output Areas 

The spatial output areas need to be ingested.

In [6]:
#set the path to the census OA directory
censusOAPath <- "../../0_data/Athens/OA/GR.Municipalities.shp"

#read the OA data
censusOA <- st_read(censusOAPath)

Reading layer `GR.Municipalities' from data source 
  `/Cities/0_data/Athens/OA/GR.Municipalities.shp' using driver `ESRI Shapefile'
Simple feature collection with 325 features and 14 fields
Geometry type: MULTIPOLYGON
Dimension:     XY
Bounding box:  xmin: 104040.7 ymin: 3850938 xmax: 1007943 ymax: 4624010
Projected CRS: GGRS87 / Greek Grid


## Census Read

In [7]:
#set the path to the census directory
censusPath <- "../../0_data/Athens/GreekCensus/2011/english/processed/"

## Select only the relevant data

We only require a subset of the census data for our purposes. We therefore need to extract the relevant data, then combine these to create our vulnerability indicators.


### Persons Level Data

We then get the persons level data and combine the variables together to create indicators:

In [8]:
# Total population
#get the data
totalPopData <- read.csv(paste0(censusPath, "A1602_SAM03_TB_DC_00_2011_B04_F_GR_Population Census 2011_Permanent Population by gender and marital status.csv"), check.names = FALSE)

#select the relevant rows, total, number of females
totalPop <- totalPopData[, c(1:2, 4,14), drop = FALSE]

# rename columns
totalPop <- totalPop %>%
  rename(total_pop_count = " Total",
        total_female_count = " Total.1")

In [9]:
# Education
# They left primary school, but can read and write / They completed pre-school education / They do not know how to read and write
educationData <- read.csv(paste0(censusPath, "A1602_SAM03_TB_DC_00_2011_B06_F_GR_Population Census 2011_Permanent Population by gender and level of education.csv"), check.names = FALSE)

#select the relevant rows
education <- educationData[, c(1:2, 10), drop = FALSE]

# rename columns
education <- education %>%
  rename(low_edu_attainment_count = " They left primary school, but can read and write / They completed pre-school education / They do not know how to read and write")

In [10]:
# forgein citizens
# Other countries / Without citizenship or unspecified citizenship or did not declare
citizenshipData <- read.csv(paste0(censusPath, "A1602_SAM03_TB_DC_00_2011_B09_F_GR_Population Census 2011_Permanent Population by groups of nationalities.csv"), check.names = FALSE)

#select the relevant rows
citizenship <- citizenshipData[, c(1:2, 8), drop = FALSE]

# rename columns
citizenship <- citizenship %>%
  rename(foreign_count =  ' Other countries / Without citizenship or unspecified citizenship or did not declare')

In [11]:
# unemployed # REDO - totals incorrect
# total unemployed
unemployedData <- read.csv(paste0(censusPath, "A1602_SAM04_TB_DC_00_2011_B02_F_GR_Population Census 2011_Permanent Population, by gender and occupation status.csv"), check.names = FALSE)

#select the relevant rows
unemployed <- unemployedData[, c(2, 7), drop = FALSE]

ids <- unique(unemployed$` Geographical code`)


if(exists("unemployedMunData")){
    remove("unemployedMunData")
}

for(i in ids){
    
    #get the municpality data
    munData <- unemployed %>% filter(` Geographical code` == i)
    
#     #get the total
    munTotal <- munData[1:1,]$` Total`
    
    # create a df
    munDF <-  data.frame(Geographical.code = i, total = munTotal)
    
    if(!exists("unemployedMunData")){
        unemployedMunData <- munDF
    } else {
        unemployedMunData <- rbind(unemployedMunData, munDF)
    }
   
}

# rename columns
unemployed <- unemployedMunData %>%
  rename(unemployed_count = 'total')


# head(unemployed)

# CALC PERC
if(exists("unemployedMunTotalData")){
    remove("unemployedMunTotalData")
}

unemployedTotalData <- unemployedData[, c(2, 4), drop = FALSE]

for(i in ids){
    
    #get the municpality data
    munTotalData <- unemployedTotalData %>% filter(` Geographical code` == i)
    
#     #get the total
    munTotal <- munTotalData[1:1,]$` Total`
    
    # create a df
    munTotalDF <-  data.frame(Geographical.code = i, total = munTotal)
    
    if(!exists("unemployedMunTotalData")){
        unemployedMunTotalData <- munTotalDF
    } else {
        unemployedMunTotalData <- rbind(unemployedMunTotalData, munTotalDF)
    }
   
}

unemployed_perc <- merge(unemployedMunTotalData, unemployed, by = 'Geographical.code')
unemployed_perc$unemployed_perc <- round((unemployed_perc$unemployed_count/unemployed_perc$total)*100,2)

unemployed_perc <- unemployed_perc %>% select(c('Geographical.code','unemployed_perc'))
names(unemployed_perc) <-  c(' Geographical level', 'unemployed_pct')

In [12]:
#employment area
#A. AGRICULTURE, FORESTRY AND FISHERIES
#F. CONSTRUCTIONS

employmentAreaData <- read.csv(paste0(censusPath, "A1602_SAM04_TB_DC_00_2011_B04_F_GR_Population Census 2011_Employed by sector of economic activity (single digit).csv"), check.names = FALSE)

#select the relevant rows
employmentArea <- employmentAreaData[, c(1:2,5,6), drop = FALSE]

# rename columns
employmentArea <- employmentArea %>%
  rename(agriculture_count =  'A. AGRICULTURE, FORESTRY AND FISHERIES',
         construction_count =  ' F. CONSTRUCTIONS')

# CALC PERC
#select the relevant rows
employmentAreaPerc <- employmentAreaData[, c(1:2,4,5,6), drop = FALSE]

# rename columns
employmentArea_perc <- employmentAreaPerc %>%
  rename(agriculture_count =  'A. AGRICULTURE, FORESTRY AND FISHERIES',
         construction_count =  ' F. CONSTRUCTIONS')

employmentArea_perc$agriculture_perc <- round((employmentArea_perc$agriculture_count/employmentArea_perc$` Total`)*100,2)
employmentArea_perc$construction_perc <- round((employmentArea_perc$construction_count/employmentArea_perc$` Total`)*100,2)


employmentArea_perc <- employmentArea_perc %>% select(c(' Geographical code','agriculture_perc', 'construction_perc'))
names(employmentArea_perc) <- c(' Geographical level', 'agriculture_pct', 'construction_pct')

In [13]:
# profession
# ' 6. Specialized farmers, stockbreeders, foresters and fishermen'
#' 9. Unskilled workers, manual workers and small professionals'
professionData <- read.csv(paste0(censusPath, "A1602_SAM04_TB_DC_00_2011_B07_F_GR_Population Census 2011_Employed by profession (single digit).csv"), check.names = FALSE)

#select the relevant rows
profession <- professionData[, c(1:2,10,13), drop = FALSE]

# rename columns
profession <- profession %>%
  rename(agr_for_fish_count =  ' 6. Specialized farmers, stockbreeders, foresters and fishermen',
         unskilled_count =  ' 9. Unskilled workers, manual workers and small professionals')

# CALC PERC
#select the relevant rows
profession_perc <- professionData[, c(1:2,4,10,13), drop = FALSE]

# rename columns
profession_perc <- profession_perc %>%
  rename(agr_for_fish_count =  ' 6. Specialized farmers, stockbreeders, foresters and fishermen',
         unskilled_count =  ' 9. Unskilled workers, manual workers and small professionals')

profession_perc$agr_for_fish_pct<- round((profession_perc$agr_for_fish_count/profession_perc$` Total`)*100,2)
profession_perc$unskilled_pct <- round((profession_perc$unskilled_count/profession_perc$` Total`)*100,2)

profession_perc <- profession_perc %>% select(c(' Geographical code','agr_for_fish_pct', 'unskilled_pct'))
names(profession_perc) <- c(' Geographical level', 'agr_for_fish_pct', 'unskilled_pct')

# profession_perc

In [14]:
# new to area
newToAreaData <- read.csv(paste0(censusPath, "A1602_SAM07_TB_DC_00_2011_B06_F_GR_Population Census 2011_Permanent population aged one year and over, by place of residence one year before the census.csv"), check.names = FALSE)

newToArea <- newToAreaData[, c(1:2, 6), drop = FALSE]

# # rename columns
newToArea <- newToArea %>%
  rename(new_to_area_count = 'In a municipality other than permanent residence or in a foreign country')

# CALC PERC
newToArea_perc <- newToAreaData[, c(1:2, 4,6), drop = FALSE]

newToArea_perc <- newToArea_perc %>%
  rename(new_to_area_count = 'In a municipality other than permanent residence or in a foreign country')

newToArea_perc$new_to_area_pct <- round((newToArea_perc$new_to_area_count/newToArea_perc$` Total`)*100,2)

newToArea_perc <- newToArea_perc %>% select(c(' Geographic Code','new_to_area_pct'))
names(newToArea_perc) <- c(' Geographical level', 'new_to_area_pct')

In [15]:
# COMBINE ALL PERSONS DATA
# education
# citizenship
# unemployed
# employmentArea
# profession
# newToArea

personsAll <- merge(totalPop, education, by = ' Geographic Code')
personsAll <- merge(personsAll, citizenship, by.x = ' Geographic Code', by.y = ' Geographical code')
personsAll <- merge(personsAll, unemployed, by.x = ' Geographic Code', by.y = 'Geographical.code')
personsAll <- merge(personsAll, employmentArea, by.x = ' Geographic Code', by.y = ' Geographical code')
personsAll <- merge(personsAll, profession, by.x = ' Geographic Code', by.y = ' Geographical code')
personsAll <- merge(personsAll, newToArea, by.x = ' Geographic Code', by.y = ' Geographic Code')

#trim the columns 
personsAll <- subset(personsAll, select=c(' Geographic Code','total_pop_count','total_female_count','low_edu_attainment_count','foreign_count','unemployed_count','agriculture_count','construction_count','agr_for_fish_count','unskilled_count','new_to_area_count'))


Warning message in merge.data.frame(personsAll, profession, by.x = " Geographic Code", :
“column names ‘ Geographical level.x’, ‘ Geographical level.y’ are duplicated in the result”
Warning message in merge.data.frame(personsAll, newToArea, by.x = " Geographic Code", :
“column names ‘ Geographical level.x’, ‘ Geographical level.y’ are duplicated in the result”


In [16]:
# calculate the calculate the percentages

personsAllPct <- personsAll

for(col in names(personsAllPct)[3:ncol(personsAllPct)]) {
  personsAllPct[paste0(col, "_pct")] = round((personsAllPct[col] / personsAllPct$total_pop_count)*100,2)
}

personsAllPct <- subset(personsAllPct, select=c(' Geographic Code','total_female_count_pct','low_edu_attainment_count_pct','foreign_count_pct'))

personsAllPct <- merge(personsAllPct, unemployed_perc, by.x = ' Geographic Code', by.y = ' Geographical level')
personsAllPct <- merge(personsAllPct, employmentArea_perc, by.x = ' Geographic Code', by.y = ' Geographical level')
personsAllPct <- merge(personsAllPct, profession_perc, by.x = ' Geographic Code', by.y = ' Geographical level')
personsAllPct <- merge(personsAllPct, newToArea_perc, by.x = ' Geographic Code', by.y = ' Geographical level')

## Households


In [17]:
# HOUSEHOLD SIZE
# 44m or less per inhabitant
if(exists("allData")){
    remove("allData")
}

sizeData <- read.csv(paste0(censusPath, "A1602_SAM05_TB_DC_00_2011_B02_F_GR_Population-Housing Census 2011_Normal dwellings by density and type of ownership.csv"))

# filter out
sizeData <- sizeData %>% filter(X.Geographical.level == 5)

# get the ids of the municipalities
ids <- unique(sizeData$X.Geographical.code)

for(i in ids){
    
    #get the municpality data
    munData <- sizeData %>% filter(X.Geographical.code == i)
    
    #get the sum of households less than 44
    munSum <- sum(munData[2:4,]$X.Total)
    
    # create a df
    munSumDF <-  data.frame(Geographical.code = i, households_44m = munSum)
    
    if(!exists("allData")){
        allData <- munSumDF
    } else {
        allData <- rbind(allData, munSumDF)
    }
   
}

names(allData) <- c(' Geographical level', 'households_44m_count')
householdSize <- allData

if(exists("allTotalData")){
    remove("allTotalData")
}
#CALC PERC
for(i in ids){
    
    #get the municpality data
    munData <- sizeData %>% filter(X.Geographical.code == i)
    
    #get the total
    munTotal <- munData[1:1,]$X.Total
    
    # create a df
    munTotalDF <-  data.frame(Geographical.code = i, total = munTotal)
    
    if(!exists("allTotalData")){
        allTotalData <- munTotalDF
    } else {
        allTotalData <- rbind(allTotalData, munTotalDF)
    }
   
}

householdSizePerc <- merge(allTotalData,allData, by.x = 'Geographical.code' , by.y = ' Geographical level')
householdSizePerc$households_44m_pct <- round((householdSizePerc$households_44m_count/householdSizePerc$total)*100,2)

householdSizePerc <- householdSizePerc %>% select(c('Geographical.code','households_44m_pct'))
names(householdSizePerc) <-  c(' Geographical level', 'households_44m_pct')

In [18]:
# no bathroom
bathroomData <- read.csv(paste0(censusPath, "A1602_SAM05_TB_DC_00_2011_B06_F_GR_Population-Housing Census 2011_Normal houses by bathroom availability.csv"), check.names = FALSE)

bathroom <- bathroomData[, c(1:2,6), drop = FALSE]

# rename columns
bathroom <- bathroom %>%
  rename(no_bathroom_count =  'There is a bathroom outside the house / There is no bath or shower')

bathroom <- bathroom %>% select(-c(' Geographical level'))

#CALC PERC
bathroomTotal <- bathroomData[, c(2,4), drop = FALSE]

bathroomPerc <- merge(bathroomTotal,bathroom)
names(bathroomPerc) <- c(' Geographical code', 'total', 'no_bathroom_count')
bathroomPerc$no_bathroom_pct <- round((bathroomPerc$no_bathroom_count/bathroomPerc$total)*100,2)

bathroomPerc <- bathroomPerc %>% select(c(' Geographical code','no_bathroom_pct'))

In [19]:
# no toilets/wc

toiletData <- read.csv(paste0(censusPath, "A1602_SAM05_TB_DC_00_2011_B08_F_GR_Population-Housing Census 2011_Normal dwellings with toilet or WC availability.csv"), check.names = FALSE)

toilet <- toiletData[, c(1:2, 7), drop = FALSE]

# rename columns
toilet <- toilet %>%
  rename(no_wc_count =  ' Has a toilet or WC (with or without plumbing) outside the house / No toilet or WC')

toilet <- toilet %>% select(-c(' Geographical level'))
#CALC PERC

toiletTotal <- toiletData[, c(2,4), drop = FALSE]

toiletPerc <- merge(toiletTotal,toilet)
names(toiletPerc) <- c(' Geographical code', 'total', 'no_toilet_count')

toiletPerc$no_toilet_pct <- round((toiletPerc$no_toilet_count/toiletPerc$total)*100,2)

toiletPerc <- toiletPerc %>% select(c(' Geographical code','no_toilet_pct'))

In [20]:
# house type
# Apartment building

houseTypeData <- read.csv(paste0(censusPath, "A1602_SAM05_TB_DC_00_2011_B14_F_GR_Population-Housing Census 2011_Normal houses by building type.csv"), check.names = FALSE)

houseType <- houseTypeData[, c(1:2, 5), drop = FALSE]

# rename columns
houseType <- houseType %>%
  rename(detached_count = 'Detached house')

houseType <- houseType %>% select(-c(' Geographical level'))

# #CALC PERC

houseTypeTotal <- houseTypeData[, c(2,4), drop = FALSE]

houseTypePerc <- merge(houseTypeTotal,houseType)
names(houseTypePerc) <- c(' Geographical code', 'total', 'detached_count')

houseTypePerc$detached_pct <- round((houseTypePerc$detached_count/houseTypePerc$total)*100,2)

houseTypePerc <- houseTypePerc %>% select(c(' Geographical code','detached_pct'))

In [21]:
# Building age
#  Before 1945	 1946 - 1960	 1961 - 1970	 1971 - 1980	 1981 - 1990	 1991 - 2000	 2001 - 2005	 2006 onwards

buildAgeData <- read.csv(paste0(censusPath, "A1602_SAM05_TB_DC_00_2011_B15_F_GR_Population-Housing Census 2011_Normal houses by construction period.csv"), check.names = FALSE)

buildAge <- buildAgeData[, c(1:2, 5), drop = FALSE]

# rename columns
buildAge <- buildAge %>%
  rename(pre_1945_count = ' Before 1945')

buildAge <- buildAge %>% select(-c(' Geographical level'))

#CALC PERC
buildAgeTotal <- buildAgeData[, c(2,4), drop = FALSE]

buildAgePerc <- merge(buildAgeTotal,buildAge)
names(buildAgePerc) <- c(' Geographical code', 'total', 'pre_1945_count')

buildAgePerc$pre_1945_pct <- round((buildAgePerc$pre_1945_count/buildAgePerc$total)*100,2)

buildAgePerc <- buildAgePerc %>% select(c(' Geographical code','pre_1945_pct'))

In [22]:
# number of rooms
#one room

roomsData <- read.csv(paste0(censusPath, "A1602_SAM05_TB_DC_00_2011_B16_F_GR_Population-Residents Census 2011_Normal houses by number of rooms.csv"), check.names = FALSE)

rooms <- roomsData[, c(1:2, 5), drop = FALSE]

# rename columns
rooms <- rooms %>%
  rename(one_room_count = '1')

rooms <- rooms %>% select(-c(' Geographical level'))

#CALC PERC
roomsTotal <- roomsData[, c(2,4), drop = FALSE]

roomsPerc <- merge(roomsTotal,rooms)
names(roomsPerc) <- c(' Geographical code', 'total', 'one_room_count')

roomsPerc$one_room_pct <- round((roomsPerc$one_room_count/roomsPerc$total)*100,2)

roomsPerc <- roomsPerc %>% select(c(' Geographical code','one_room_pct'))

In [23]:
# small area households
# <40 m2

smallHouseholdsData <- read.csv(paste0(censusPath, "A1602_SAM05_TB_DC_00_2011_B17_F_GR_Population-Residents Census 2011_Normal dwellings by area (m 2 ).csv"), check.names = FALSE)

smallHouseholds <- smallHouseholdsData[, c(1:2, 5), drop = FALSE]

# rename columns
smallHouseholds <- smallHouseholds %>%
  rename(less_40m2_count = ' Under 40')


smallHouseholds <- smallHouseholds %>% select(-c(' Geographical level'))

#CALC PERC
smallHouseholdsTotal <- smallHouseholdsData[, c(2,4), drop = FALSE]

smallHouseholdsPerc <- merge(smallHouseholdsTotal,smallHouseholds)
names(smallHouseholdsPerc) <- c(' Geographical code', 'total', 'less_40m2_count')

smallHouseholdsPerc$less_40m2_pct <- round((smallHouseholdsPerc$less_40m2_count/smallHouseholdsPerc$total)*100,2)

smallHouseholdsPerc <- smallHouseholdsPerc %>% select(c(' Geographical code','less_40m2_pct'))

In [24]:
# insulation
# No insulation at all
insulationData <- read.csv(paste0(censusPath, "A1602_SAM05_TB_DC_00_2011_B29_F_GR_Population Census 2011_Normal houses by type of insulation.csv"), check.names = FALSE)

insulation <- insulationData[, c(1:2, 8), drop = FALSE]

# rename columns
insulation <- insulation %>%
  rename(no_insulation_count = ' No insulation at all')
insulation <- insulation %>% select(-c(' Geographical level'))

#CALC PERC
insulationTotal <- insulationData[, c(2,4), drop = FALSE]

insulationPerc <- merge(insulationTotal,insulation)
names(insulationPerc) <- c(' Geographical code', 'total', 'no_insulation_count')

insulationPerc$no_insulation_pct <- round((insulationPerc$no_insulation_count/insulationPerc$total)*100,2)

insulationPerc <- insulationPerc %>% select(c(' Geographical code','no_insulation_pct'))

In [25]:
# no car
carData <- read.csv(paste0(censusPath, "A1602_SAM05_TB_DC_00_2011_B36_F_GR_Population Census 2011_Households by number of cars at their disposal.csv"), check.names = FALSE)

car <- carData[, c(1:2, 5), drop = FALSE]

# # rename columns
car <- car %>%
  rename(no_car_count = ' 0 cars')

car <- car %>% select(-c(' Geographical level'))

#CALC PERC
carTotal <- carData[, c(2,4), drop = FALSE]

carPerc <- merge(carTotal,car)
names(carPerc) <- c(' Geographical code', 'total', 'no_car_count')

carPerc$no_car_pct <- round((carPerc$no_car_count/carPerc$total)*100,2)

carPerc <- carPerc %>% select(c(' Geographical code','no_car_pct'))

In [26]:
# no internet access

internetAccessData <- read.csv(paste0(censusPath, "A1602_SAM05_TB_DC_00_2011_B44_F_GR_Population Census 2011_Percentage distribution of households according to internet access.csv"), check.names = FALSE)

internetAccess <- internetAccessData[, c(1:2, 6), drop = FALSE]

# rename columns
internetAccess <- internetAccess %>%
  rename(no_internet_pct = ' do not have internet access')

internetAccess <- internetAccess %>% select(-c('Geographical level'))

In [27]:
#Merge all

# counts
householdsAll <- merge(householdSize, bathroom, by.x = ' Geographical level', by.y = ' Geographical code')
householdsAll <- merge(householdsAll, toilet, by.x = ' Geographical level', by.y = ' Geographical code')
householdsAll <- merge(householdsAll, houseType, by.x = ' Geographical level', by.y = ' Geographical code')
householdsAll <- merge(householdsAll, buildAge, by.x = ' Geographical level', by.y = ' Geographical code')
householdsAll <- merge(householdsAll, rooms, by.x = ' Geographical level', by.y = ' Geographical code')
householdsAll <- merge(householdsAll, smallHouseholds, by.x = ' Geographical level', by.y = ' Geographical code')
householdsAll <- merge(householdsAll, insulation, by.x = ' Geographical level', by.y = ' Geographical code')
householdsAll <- merge(householdsAll, car, by.x = ' Geographical level', by.y = ' Geographical code')
householdsAll <- merge(householdsAll, internetAccess, by.x = ' Geographical level', by.y = ' Geographical code')

# Percentages
householdsAllPerc <- merge(householdSizePerc, bathroomPerc, by.x = ' Geographical level', by.y = ' Geographical code')
householdsAllPerc <- merge(householdsAllPerc, toiletPerc, by.x = ' Geographical level', by.y = ' Geographical code')
householdsAllPerc <- merge(householdsAllPerc, houseTypePerc, by.x = ' Geographical level', by.y = ' Geographical code')
householdsAllPerc <- merge(householdsAllPerc, buildAgePerc, by.x = ' Geographical level', by.y = ' Geographical code')
householdsAllPerc <- merge(householdsAllPerc, roomsPerc, by.x = ' Geographical level', by.y = ' Geographical code')
householdsAllPerc <- merge(householdsAllPerc, smallHouseholdsPerc, by.x = ' Geographical level', by.y = ' Geographical code')
householdsAllPerc <- merge(householdsAllPerc, insulationPerc, by.x = ' Geographical level', by.y = ' Geographical code')
householdsAllPerc <- merge(householdsAllPerc, carPerc, by.x = ' Geographical level', by.y = ' Geographical code')
householdsAllPerc <- merge(householdsAllPerc, internetAccess, by.x = ' Geographical level', by.y = ' Geographical code')

## Z-Scores

The raw data is not suitable for use within the vulnerabiltiy assessment. It needs to be standardised. Therefore, the data is converted to z-scores. Z-scores are:

>"A statistical measurement of a score's relationship to the mean (average value) in a group of scores. A Z-score of 0 means the score is the same as the mean (average value). A Z-score can be positive or negative, indicating whether it is above or below the mean and by how many standard deviations. Z-score standardisation represents the deviation of a raw score from its mean in standard deviation units." (Kazmierczak et al., 2015)

## Persons Z-scores

In [28]:
#Z scores

#Copy the data
allPersonsDataZ <- personsAllPct

#not using female data for now
allPersonsDataZ <- allPersonsDataZ[, c(1, 3:10), drop = FALSE]

# Calculate the z scores for each of the relevant columns - starting at the 2nd column
for(col in names(allPersonsDataZ)[2:ncol(allPersonsDataZ)]) {
#     print(col)
  allPersonsDataZ[paste0(col, "_Z")] = scale(allPersonsDataZ[col], )
}

# remove the original data to leave only the z scores
allPersonsDataZ <- allPersonsDataZ[-c(2:9)]
# # summary(personsDataZ)
head(allPersonsDataZ)

#output the data as a csv
write.csv(allPersonsDataZ, file.path(pipelineDir, "persons_oa_z_data.csv"), row.names = FALSE)

,Geographic Code,low_edu_attainment_count_pct_Z,foreign_count_pct_Z,unemployed_pct_Z,agriculture_pct_Z,construction_pct_Z,agr_for_fish_pct_Z,unskilled_pct_Z,new_to_area_pct_Z
,<int>,"<dbl[,1]>","<dbl[,1]>","<dbl[,1]>","<dbl[,1]>","<dbl[,1]>","<dbl[,1]>","<dbl[,1]>","<dbl[,1]>"
1,1,-0.10948913,-0.50473849,0.8589236,-0.30400733,-0.57788016,-0.27599891,-0.45745002,-0.015348417
2,2,0.09468197,-0.05414168,0.1827130,0.04616694,-0.08587229,-0.04102041,0.21414254,-0.503434659
3,3,-1.22108732,0.65432679,0.7699485,-1.13086204,-0.57788016,-1.11788048,-0.39361043,-0.035406756
4,4,-0.50421991,0.06529361,-0.2977524,-0.44290327,0.07725331,-0.41369332,0.01240941,0.171862745
5,11,-0.17527759,-0.50202405,0.9597620,-0.31574502,-0.71469518,-0.28872068,-0.42680702,-0.001976191
6,12,0.14459046,-0.51559625,0.4318432,-0.25379612,-0.01220267,-0.22137015,-0.58768280,-0.075523433


## Households Z-scores

In [29]:
#Z scores

#Copy the data
allHouseholdsDataZ <- householdsAllPerc

# Calculate the z scores for each of the relevant columns - starting at the 2nd column
for(col in names(allHouseholdsDataZ)[2:ncol(allHouseholdsDataZ)]) {
#     print(col)
  allHouseholdsDataZ[paste0(col, "_Z")] = scale(allHouseholdsDataZ[col], )
}

#remove the original data to leave only the z scores
allHouseholdsDataZ <- allHouseholdsDataZ[-c(2:ncol(householdsAllPerc))]

# invesrse the detached data
allHouseholdsDataZ$detached_pct_Z = allHouseholdsDataZ$detached_pct_Z*-1

# head(allHouseholdsDataZ)

#output the data as a csv
write.csv(allHouseholdsDataZ, file.path(pipelineDir, "households_oa_z_data.csv"), row.names = FALSE)

## Combine Data

The persons level and household level data are then combined into a single CSV:

In [30]:
#Combine the RAW persons and household data
personsHouseholdDataCombined <- merge(personsAll, householdsAll, by.x = ' Geographic Code', by.y = ' Geographical level')

#output the data as a csv
write.csv(personsHouseholdDataCombined, file.path(pipelineDir, "census_oa_raw_data.csv"), row.names = FALSE)

# # #Combine the % persons and household data
personsHouseholdPctDataCombined <-  merge(personsAllPct, householdsAllPerc, by.x = ' Geographic Code', by.y = ' Geographical level')

#output the data as a csv
write.csv(personsHouseholdPctDataCombined, file.path(pipelineDir, "census_oa_pct_data.csv"), row.names = FALSE)

# #Combine the Z-score persons and household data
personsHouseholdZDataCombined <- merge(allPersonsDataZ, allHouseholdsDataZ, by.x = ' Geographic Code', by.y = ' Geographical level')

# # names(personsHouseholdZDataCombined) <- gsub("_pct_Z","",names(personsHouseholdZDataCombined))

#output the data as a csv
write.csv(personsHouseholdZDataCombined, file.path(pipelineDir, "census_persons_oa_z_data.csv"), row.names = FALSE)

In [31]:
# spatial data

# create a new column of the last four digits of Geographic Code to allow merge with spatial data
municipalityCodes <- totalPopData %>% select(c(' Geographical level',' Geographic Code'))
names(municipalityCodes) <- c('Geographical_level',' Geographic Code')
municipalityCodes <- municipalityCodes %>% filter(municipalityCodes$Geographical_level == 5)

#short code
municipalityCodes$short_code <- str_sub(municipalityCodes$` Geographic Code`,-4)

# raw
personsHouseholdDataCombinedShortCode <- merge(personsHouseholdDataCombined, municipalityCodes, by.x = ' Geographic Code', by.y = ' Geographic Code')

# export the raw data
oaDataRaw <- merge(censusOA[c(5)], personsHouseholdDataCombinedShortCode, by.x = 'CodeELSTAT', by.y = 'short_code')
st_write(oaDataRaw, file.path(pipelineDir, "census_all_oa_raw_data.geojson"), delete_dsn = TRUE)

#pct
personsHouseholdPctDataCombinedShortCode <- merge(personsHouseholdPctDataCombined, municipalityCodes, by.x = ' Geographic Code', by.y = ' Geographic Code')

# export the raw data
oaDataPct <- merge(censusOA[c(5)], personsHouseholdPctDataCombinedShortCode, by.x = 'CodeELSTAT', by.y = 'short_code')
st_write(oaDataPct, file.path(pipelineDir, "census_all_oa_pct_data.geojson"), delete_dsn = TRUE)

#merge the z data
personsHouseholdZDataCombinedShortCode <- merge(personsHouseholdZDataCombined, municipalityCodes, by.x = ' Geographic Code', by.y = ' Geographic Code')

# export the z data
oaDataZ <- merge(censusOA[c(5)], personsHouseholdZDataCombinedShortCode, by.x = 'CodeELSTAT', by.y = 'short_code')
st_write(oaDataZ, file.path(pipelineDir, "census_all_oa_z_data.geojson"), delete_dsn = TRUE)

Deleting source `../../2_pipeline/Athens/1a_CensusData/2011/census_all_oa_raw_data.geojson' using driver `GeoJSON'
Writing layer `census_all_oa_raw_data' to data source 
  `../../2_pipeline/Athens/1a_CensusData/2011/census_all_oa_raw_data.geojson' using driver `GeoJSON'
Writing 308 features with 23 fields and geometry type Multi Polygon.
Deleting source `../../2_pipeline/Athens/1a_CensusData/2011/census_all_oa_pct_data.geojson' using driver `GeoJSON'
Writing layer `census_all_oa_pct_data' to data source 
  `../../2_pipeline/Athens/1a_CensusData/2011/census_all_oa_pct_data.geojson' using driver `GeoJSON'
Writing 308 features with 22 fields and geometry type Multi Polygon.
Deleting source `../../2_pipeline/Athens/1a_CensusData/2011/census_all_oa_z_data.geojson' using driver `GeoJSON'
Writing layer `census_all_oa_z_data' to data source 
  `../../2_pipeline/Athens/1a_CensusData/2011/census_all_oa_z_data.geojson' using driver `GeoJSON'
Writing 308 features with 21 fields and geometry type M

## Athens

In [32]:
# dervied from qgis - the codes of the area around athens 
athens <- c('4501','4502','4503','4504','4505','4506','4507','4508','4601','4602','4603','4604','4605','4606','4607','4608','4609','4610','4611','4612','4701','4702','4703','4704','4705','4706','4707','4801','4802','4803','4804','4805','4806','4807','4808','4901','4904','4908','4909','5002','5005','5101','5102','5103','5104','5105','5201')

In [33]:
# get just the athens data
oaDataPctAthens <- subset(oaDataPct, CodeELSTAT %in% athens)
oaDataPctAthensZ <- st_drop_geometry(oaDataPctAthens)

In [34]:
# Calculate the z scores for each of the relevant columns - starting at the 2nd column
for(col in names(oaDataPctAthensZ)[3:21]) {
#     print(col)
  oaDataPctAthensZ[paste0(col, "_Z")] = scale(oaDataPctAthensZ[col], )
}

# # #remove the original data to leave only the z scores
oaDataPctAthensZ <- oaDataPctAthensZ[-c(2:ncol(oaDataPctAthens))]

In [35]:
# spatial
oaDataPctAthensZ <- merge(censusOA[c(5)], oaDataPctAthensZ, by.x = 'CodeELSTAT', by.y = 'CodeELSTAT')

In [36]:
# export
st_write(oaDataPctAthensZ, file.path(pipelineDir, "census_athens_oa_z_data.geojson"), delete_dsn = TRUE)

Deleting source `../../2_pipeline/Athens/1a_CensusData/2011/census_athens_oa_z_data.geojson' using driver `GeoJSON'
Writing layer `census_athens_oa_z_data' to data source 
  `../../2_pipeline/Athens/1a_CensusData/2011/census_athens_oa_z_data.geojson' using driver `GeoJSON'
Writing 47 features with 19 fields and geometry type Multi Polygon.


**END**